# Creating Model for Skin Disease Classification using CNN

## 1. Preparation

Importing necessary library

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt

Defining path

In [2]:
train_dir = r'C:\Users\mgusn\OneDrive\Documents\alay sat\Bangkit\ML\dermnet_data\train'
test_dir = r'C:\Users\mgusn\OneDrive\Documents\alay sat\Bangkit\ML\dermnet_data\test'

Inspecting Directory Structure

In [3]:
import os

def list_directory_contents(directory, max_files_per_level=5):
    for root, dirs, files in os.walk(directory):
        level = root.replace(directory, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        file_count = 0
        for f in files:
            if file_count < max_files_per_level:
                print(f'{subindent}{f}')
                file_count += 1
            else:
                print(f'{subindent}...')
                break

# Example usage:
print("Training Directory Structure:")
list_directory_contents(train_dir)

print("\nTest Directory Structure:")
list_directory_contents(test_dir)


Training Directory Structure:
train/
    Acne and Rosacea Photos/
        0_1.jpg
        0_13.jpg
        0_14.jpg
        0_2.jpg
        0_31.jpg
        ...
    Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/
        0_12.jpg
        0_15.jpg
        0_16.jpg
        0_17.jpg
        0_18.jpg
        ...
    Atopic Dermatitis Photos/
        0_1.jpg
        0_10.jpg
        0_11.jpg
        0_12.jpg
        0_14.jpg
        ...
    Bullous Disease Photos/
        0_11.jpg
        0_14.jpg
        0_18.jpg
        0_24.jpg
        0_29.jpg
        ...
    Cellulitis Impetigo and other Bacterial Infections/
        0_1.jpg
        0_10.jpg
        0_101.jpg
        0_103.jpg
        0_105.jpg
        ...
    Eczema Photos/
        0_0.jpg
        0_16.jpg
        0_19.jpg
        0_28.jpg
        0_38.jpg
        ...
    Exanthems and Drug Eruptions/
        0_0.jpg
        0_10.jpg
        0_101.jpg
        0_102.jpg
        0_103.jpg
        ...
    Hair Loss Ph

## 2. Augmenting Dataset

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

Found 24855 images belonging to 23 classes.
Found 4317 images belonging to 23 classes.


## 3. Creating Model Architecture

Load Pre-trained model

In [5]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

C:\Users\mgusn\AppData\Local\Temp\ipykernel_21388\3880497335.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


Creating the Model

In [7]:
# Create a new Sequential model
model = Sequential()

# Add MobileNetV2 base
model.add(base_model)

# Add GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Add custom layers
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='softmax'))


Compile the Model

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Training the Model

In [9]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=100  
)

Epoch 1/100


c:\Users\mgusn\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 11/389 ━━━━━━━━━━━━━━━━━━━━ 7:02 1s/step - accuracy: 0.0580 - loss: 4.1757

## 4. Evaluating Model

Validation and Training Comparison

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

Try to predict with the model

In [ ]:
# Get an image from the validation dataset
img_batch, true_labels_batch = next(test_generator)
img = img_batch[0]
true_label_index = np.argmax(true_labels_batch[0])

# Display the image
plt.imshow(img)
plt.axis('off')
plt.show()

# Make prediction
prediction_scores = model.predict(np.expand_dims(img, axis=0))
predicted_label_index = np.argmax(prediction_scores)

# Get the true and predicted class labels
true_label = test_generator.classes[true_label_index]
predicted_label = test_generator.classes[predicted_label_index]

print("True label: " + test_generator.class_names[true_label])
print("Predicted label: " + test_generator.class_names[predicted_label])


## 5. Save Model

Save model as HDF5 (.h5)

In [ ]:
model.save("skin_disease_classifier.h5")

Convert model to JSON and save

In [ ]:
model_json = model.to_json()
with open("skin_disease_classifier.json", "w") as json_file:
    json_file.write(model_json)

Save model as TFlite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("skin_disease_classifier.tflite", "wb") as tflite_file:
    tflite_file.write(tflite_model)